# Introduction/Business Problem

Nowadays, with the necessity of accommodation, either for traveling purpose or for business trips, hotels became more and more needed in every city in the world, especially those with more tourist attractions or those have hundreds of thousands of companies of all sizes. Here, the city, Seattle, is one of such cities. It is now developing very fast and many word famous companies are settled there, like Google and Microsoft. It is also a beautiful city, with many tourists every year. Therefore, hotels became a necessity in that city. With increasing number of companies and visitors, it is a good opportunity to invest in a hotel in that city. But where to choose to open a new hotel becomes a problem. You know, in some popular places, there is sure to have many mature hotels. And a new hotel may not be as popular as the existing ones and that could lead to a failure in investment. So here we would use data to make decisions where should be a better place for this investment. This project could also be interested by those who would like to make investments as well.

# Data

I used two parts of data in this project. One is the neighbourhood data of Seattle, which was scraped from Wikipedia using the BeautifulSoup. The data contains the neighbourhoods inside the whole Seattle Area and the bigger district it belongs to. As I would only consider invest the hotel in the center area of Seattle, so I filter out other areas which are far away from the center. The other part of the data is extracted from Foursquare. It is the data of different venues around Seattle, which include coffee shops, bars, restaurants, hotels, etc. I will use the data to see what kinds of venues are in each neighbourhood and decide where is the new hotel needed.

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Solving environment: ...working... done

# All requested packages already installed.



usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [2]:
#from whichwebsite to scrap table
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle').text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of neighborhoods in Seattle - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XjsEGwpAIC8AAEsv1EwAAAFI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_Seattle","wgTitle":"List of neighborhoods in Seattle","wgCurRevisionId":931322291,"wgRevisionId":931322291,"wgArticleId":750943,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive

In [4]:
table=soup.find('tbody')
table

<tbody><tr>
<th width="2%">
</th>
<th width="12%">Neighborhood name
</th>
<th width="12%">Within larger district
</th>
<th width="7%">Annexed<sup class="reference" id="cite_ref-41"><a href="#cite_note-41">[41]</a></sup>
</th>
<th class="unsortable">Locator map
</th>
<th class="unsortable">Street map
</th>
<th class="unsortable">Image
</th>
<th class="unsortable">Notes
</th></tr>
<tr>
<th><small>1</small>
</th>
<td><b>North Seattle</b>
</td>
<td>Seattle
</td>
<td>Various
</td>
<td style="text-align:center;"><a class="image" href="/wiki/File:Norda_Seatlo.jpg"><img alt="Norda Seatlo.jpg" data-file-height="599" data-file-width="338" decoding="async" height="142" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Norda_Seatlo.jpg/80px-Norda_Seatlo.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Norda_Seatlo.jpg/120px-Norda_Seatlo.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Norda_Seatlo.jpg/160px-Norda_Seatlo.jpg 2x" width="80"/></a>
</td>
<td style="

In [5]:
A=[]
B=[]

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==7:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))

In [6]:
df=pd.DataFrame(B,columns=['District'])
df['Neighbourhood']=A
df.head()

,District,Neighbourhood
0,Seattle,North Seattle
1,North Seattle,Broadview
2,North Seattle,Bitter Lake
3,North Seattle,North Beach
4,North Seattle,Crown Hill


In [7]:
seattle=df[df['District'].str.contains('Seattle')]
seattle.head()

,District,Neighbourhood
0,Seattle,North Seattle
1,North Seattle,Broadview
2,North Seattle,Bitter Lake
3,North Seattle,North Beach
4,North Seattle,Crown Hill


In [8]:
#define a function to get latitude and longtitude by postcode
def get_geocoder(neighbourhood):
    lat_lng_coords=None
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{}, Seattle, USA'.format(neighbourhood.strip()))
        lat_lng_coords=g.latlng
        latitude=lat_lng_coords[0]
        longitude=lat_lng_coords[1]
    return latitude,longitude

In [9]:
#add latitude and longtitude to the dataframe
seattle['Latitude'], seattle['Longitude']=zip(*seattle['Neighbourhood'].apply(get_geocoder))
seattle.head()

C:\Users\kary-\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,District,Neighbourhood,Latitude,Longitude
0,Seattle,North Seattle,47.643727,-122.302939
1,North Seattle,Broadview,47.722380,-122.364980
2,North Seattle,Bitter Lake,47.718680,-122.350300
3,North Seattle,North Beach,47.700718,-122.389593
4,North Seattle,Crown Hill,47.695200,-122.374100


In [10]:
#create a map for Toronto
address='Seattle, USA'
geolocator=Nominatim(user_agent="Seattle_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geograpical coordinate of Seattle are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seattle are 47.6038321, -122.3300624.


In [11]:
map_Seattle=folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(seattle['Latitude'], seattle['Longitude'], seattle['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Seattle)  
    
map_Seattle

# Methodology

First, scrap the data from Wikipedia and transformed it into dataframe. As only the data in the seattle area is needed, filter the data and keep only the disricts which include "Seattle". Then use get geo_coder to get the latitude and longtitude of each neighbourhood, and then create the map of Seattle.  
Second, obtain venue data from Foursquare for different neighbourhoods. There came the result that only 6 neighbourhoods have more than 90 venues nearby. Therefore only use these 6 neighbourhoods because hotels should always be in somewhere popular. Then explore the categories of venues and decide which neighbourhood could be a good place for this investment.

In [12]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'SVSCM14TUGHYRHDIY0ZKGQYIMBCHJYCVJBFYPN3WIFFNSENT' # your Foursquare ID
CLIENT_SECRET = 'J0F2SOLBUJ0SSNLBHTJDAISAEPGKH04YEMUZZG31WAKFE0WC' # your Foursquare Secret
VERSION = '20200208' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SVSCM14TUGHYRHDIY0ZKGQYIMBCHJYCVJBFYPN3WIFFNSENT
CLIENT_SECRET:J0F2SOLBUJ0SSNLBHTJDAISAEPGKH04YEMUZZG31WAKFE0WC


In [16]:
from pandas.io.json import json_normalize

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
Seattle_venues = getNearbyVenues(names=seattle['Neighbourhood'],
                                 latitudes=seattle['Latitude'],
                                 longitudes=seattle['Longitude']
                                  )

North Seattle
Broadview
Bitter Lake
North Beach
Crown Hill
Greenwood
Northgate
Lake City
Wedgwood
View Ridge
Sand Point
Roosevelt
Ravenna
Bryant
Windermere
Laurelhurst
University District
Wallingford
Green Lake
Fremont
Phinney Ridge
Ballard
Central Seattle
Magnolia
Interbay
Queen Anne
Capitol Hill
Lake Union
Downtown
Central Area
South End
West Seattle
Alki Point
North Admiral
Junction
Seaview
Fairmount Park
Genesee
Gatewood
Fauntleroy
Arbor Heights
Delridge


In [21]:
Seattle_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Seattle,47.643727,-122.302939,Seattle Public Library - Montlake,47.640520,-122.302413,Library
1,North Seattle,47.643727,-122.302939,Montlake Cut,47.647094,-122.304686,Canal
2,North Seattle,47.643727,-122.302939,Cafe Lago,47.639698,-122.302256,Italian Restaurant
3,North Seattle,47.643727,-122.302939,Fuel Coffee - Montlake,47.639688,-122.302009,Coffee Shop
4,North Seattle,47.643727,-122.302939,Montlake Blvd Market,47.643480,-122.303915,Grocery Store


In [22]:
#check how many venues were returned for each neighbourhood
Seattle_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Alki Point,5,5,5,5,5,5
Arbor Heights,1,1,1,1,1,1
Ballard,97,97,97,97,97,97
Bitter Lake,27,27,27,27,27,27
Broadview,2,2,2,2,2,2
Bryant,6,6,6,6,6,6
Capitol Hill,55,55,55,55,55,55
Central Area,4,4,4,4,4,4
Central Seattle,100,100,100,100,100,100


In [34]:
#choose those neighbourhoods whose number of nearby is greater than 90
big_neighbourhoods=Seattle_venues[Seattle_venues['Neighbourhood'].isin(['Ballard','Central Seattle','Downtown','Lake Union','University District','West Seattle'])]
big_neighbourhoods.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
203,University District,47.66127,-122.31307,Sweet Alchemy Ice Cream,47.659877,-122.313242,Ice Cream Shop
204,University District,47.66127,-122.31307,Sharetea,47.661370,-122.316245,Bubble Tea Shop
205,University District,47.66127,-122.31307,Pink Gorilla,47.661026,-122.313238,Video Game Store
206,University District,47.66127,-122.31307,Shawarma King,47.660923,-122.313199,Mediterranean Restaurant
207,University District,47.66127,-122.31307,Neptune Theatre,47.661105,-122.313995,Music Venue


In [37]:
big_neighbourhoods=big_neighbourhoods.reset_index()
big_neighbourhoods=big_neighbourhoods.drop(['index'],axis=1)
big_neighbourhoods.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,University District,47.66127,-122.31307,Sweet Alchemy Ice Cream,47.659877,-122.313242,Ice Cream Shop
1,University District,47.66127,-122.31307,Sharetea,47.661370,-122.316245,Bubble Tea Shop
2,University District,47.66127,-122.31307,Pink Gorilla,47.661026,-122.313238,Video Game Store
3,University District,47.66127,-122.31307,Shawarma King,47.660923,-122.313199,Mediterranean Restaurant
4,University District,47.66127,-122.31307,Neptune Theatre,47.661105,-122.313995,Music Venue


In [38]:
#find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(big_neighbourhoods['Venue Category'].unique())))

There are 148 uniques categories.


In [39]:
# one hot encoding
Seattle_onehot=pd.get_dummies(big_neighbourhoods[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
Seattle_onehot['Neighbourhood']=big_neighbourhoods['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns=[Seattle_onehot.columns[-1]]+list(Seattle_onehot.columns[:-1])
Seattle_onehot=Seattle_onehot[fixed_columns]

Seattle_onehot.head()

,Neighbourhood,ATM,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,...,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,University District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,University District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,University District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,University District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,University District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
Seattle_grouped=Seattle_onehot.groupby('Neighbourhood').mean().reset_index()
Seattle_grouped.head()

,Neighbourhood,ATM,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,...,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Ballard,0.00,0.000000,0.00,0.000000,0.0,0.00,0.00,0.00,0.020619,...,0.010309,0.00,0.00,0.00,0.010309,0.00,0.020619,0.00,0.0,0.010309
1,Central Seattle,0.00,0.000000,0.00,0.000000,0.0,0.00,0.00,0.00,0.020000,...,0.010000,0.00,0.01,0.00,0.010000,0.00,0.020000,0.01,0.0,0.010000
2,Downtown,0.00,0.000000,0.01,0.000000,0.0,0.00,0.02,0.00,0.010000,...,0.010000,0.01,0.01,0.00,0.000000,0.00,0.000000,0.01,0.0,0.000000
3,Lake Union,0.00,0.010417,0.00,0.020833,0.0,0.00,0.00,0.00,0.010417,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.010417,0.00,0.0,0.010417
4,University District,0.01,0.000000,0.00,0.010000,0.0,0.01,0.03,0.01,0.000000,...,0.030000,0.00,0.00,0.01,0.000000,0.01,0.040000,0.00,0.0,0.020000


In [41]:
#write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues=10

indicators=['st', 'nd', 'rd']

# create columns according to number of top venues
columns=['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted=pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood']=Seattle_grouped['Neighbourhood']

for ind in np.arange(Seattle_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:]=return_most_common_venues(Seattle_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballard,Mexican Restaurant,Bar,Coffee Shop,Cocktail Bar,Burger Joint,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,New American Restaurant,Clothing Store
1,Central Seattle,Coffee Shop,Bar,Mexican Restaurant,Cocktail Bar,Ice Cream Shop,Sushi Restaurant,New American Restaurant,Brewery,Seafood Restaurant,Clothing Store
2,Downtown,Coffee Shop,Cocktail Bar,Café,Hotel,Donut Shop,Italian Restaurant,Scenic Lookout,Historic Site,Indian Restaurant,Japanese Restaurant
3,Lake Union,Coffee Shop,Café,Bakery,Hotel,Food Truck,Bar,Pizza Place,Exhibit,Gym / Fitness Center,Restaurant
4,University District,Coffee Shop,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant,Asian Restaurant,Café,Thrift / Vintage Store,Sandwich Place
5,West Seattle,Hotel,Coffee Shop,American Restaurant,Clothing Store,Café,Italian Restaurant,New American Restaurant,Gym,Restaurant,Japanese Restaurant


In [46]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters=2
Seattle_grouped_clustering=Seattle_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(Seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 0, 0])

In [51]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Seattle_merged=big_neighbourhoods

# merge toronto_grouped with neighbourhood to add latitude/longitude for each neighbourhood
Seattle_merged=Seattle_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Seattle_merged.head() # check the last columns!

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,University District,47.66127,-122.31307,Sweet Alchemy Ice Cream,47.659877,-122.313242,Ice Cream Shop,0,Coffee Shop,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant,Asian Restaurant,Café,Thrift / Vintage Store,Sandwich Place
1,University District,47.66127,-122.31307,Sharetea,47.661370,-122.316245,Bubble Tea Shop,0,Coffee Shop,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant,Asian Restaurant,Café,Thrift / Vintage Store,Sandwich Place
2,University District,47.66127,-122.31307,Pink Gorilla,47.661026,-122.313238,Video Game Store,0,Coffee Shop,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant,Asian Restaurant,Café,Thrift / Vintage Store,Sandwich Place
3,University District,47.66127,-122.31307,Shawarma King,47.660923,-122.313199,Mediterranean Restaurant,0,Coffee Shop,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant,Asian Restaurant,Café,Thrift / Vintage Store,Sandwich Place
4,University District,47.66127,-122.31307,Neptune Theatre,47.661105,-122.313995,Music Venue,0,Coffee Shop,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant,Asian Restaurant,Café,Thrift / Vintage Store,Sandwich Place


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Seattle_merged['Neighbourhood Latitude'], Seattle_merged['Neighbourhood Longitude'], Seattle_merged['Neighbourhood'], Seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Results and Discussion

The results could be seen in two parts:  
1. Among the ten most common venue categories, hotel does not appear in Ballard, Central Seattle and University District. As common sense, there is not much necessity of hotels in university districts because there are neither tourist attractions nor companies there which will require accommodation for short business visits. 
2. Using the cluster model, the result shows that only Ballard and Central Seattle are in the same cluster, while others are in the other cluster. This also proves that there should be more need for hotels in these two neighbourhoods.

# Conclusion

Based on our analysis with the data from Foursquare, we found two neighbourhoods of six big neighbourhoods in Seattle have the necessity in hotel. As there are already many hotels in other four big neighbourhoods, invest hotels in those neighbourhoods may lead to small profits or even loss because the competition there has already been aggresive. It's a wise decision to invest in Ballard and Central Seattle because as a businessman, the aim is to make money. And investment in these two neighbourhoods has more probability to bring profits. 